# Projeto Aprendizado de Máquina

## CMC-13 Introdução a Ciência de Dados
## Equipe:
### André Luiz de Melo Thiessen
### Nikollas da Silva Antes
### Pedro Anacleto Martins Senna de Oliveira

## Importando bibliotecas

In [83]:
#Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

## Importando os datasets

In [84]:
#Importando as bases de dados
movies = pd.read_csv("movies.csv", on_bad_lines='skip', sep = ';')
ratings = pd.read_csv("ratings.csv", sep = ';')
users = pd.read_csv("users.csv")

## Data Cleaning
- Primeiramente, foi verificado a existência de valores nulos (dados faltantes)

In [85]:
# Verificando a existência de valores nulos (dados faltantes)
print(movies.isnull().values.any())
print(ratings.isnull().values.any())
print(users.isnull().values.any())

False
False
False


- Depois foram apagadas as colunas que julgou-se impertinentes para o aprendizado em questão
    - Title, Timestamp, Zip-code e name: irrelevantes
    - Genres dos movies: Deixaria a árvore muito grande, optamos por retirar tal atributo

In [86]:
#Apagando as colunas irrelevantes
movies = movies.drop(columns='Title')
ratings = ratings.drop(columns='Timestamp')
users = users.drop(columns='Zip-code')
users = users.drop(columns='name')
movies = movies.drop(columns='Genres')

## Transformações de Dados

- Foi alterado a data de nascimento (birthday) para o padrão proposto no lab, agrupando em idades

In [87]:
format_string = "%m/%d/%Y"
def calculateAge(birthDate): 
    today = date.today() 
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
    return age

for i in users.index:
    [month, day, year] = users['birthday'][i].split('/')
    if(day == '0'):
        users.loc[i, 'birthday'] = month+'/'+'1/'+year
    date = datetime.strptime(users['birthday'][i], format_string)
    age = calculateAge(date)
    users.loc[i, 'birthday'] = age
    
    #Agrupando os usuários em 
for i in users.index:
    if(int(users['birthday'][i])<18):
        users.loc[i, 'birthday'] = '1'
    elif (int(users['birthday'][i])<24):
        users.loc[i, 'birthday'] = '18'
    elif (int(users['birthday'][i])<35):
        users.loc[i, 'birthday'] = '25'
    elif (int(users['birthday'][i])<45):
        users.loc[i, 'birthday'] = '35'
    elif (int(users['birthday'][i])<50):
        users.loc[i, 'birthday'] = '45'
    elif (int(users['birthday'][i])<55):
        users.loc[i, 'birthday'] = '50'
    else:
        users.loc[i, 'birthday'] = '56'


- Além disso, fez-se um merge das tabelas para facilitar o trabalho futuro
- Nomeou-se o novo dataframe de df

In [88]:
df = pd.merge(ratings,users)
df = pd.merge(df,movies)
df = df.drop(columns='UserID')
display(df)

,MovieID,Rating,Gender,Occupation,birthday
0,1193,5,F,10,1
1,1193,5,M,16,56
2,1193,4,M,12,25
3,1193,4,M,7,25
4,1193,5,M,1,50
...,...,...,...,...,...
1000176,2198,5,M,17,1
1000177,2703,3,M,14,35
1000178,2845,1,M,17,18
1000179,3607,5,F,20,18


## Árvore de decisão

### Cálculo da entropia

In [89]:
def entropy(df):
    rating = df.columns[1]
    values, counts = np.unique(df[rating], return_counts = True)
    prob = counts/(counts.sum())
    entropy = sum(-prob*np.log2(prob))
    return entropy

entropy(df)

2.1002336337757876

### Cálculo do ganho de informação

In [90]:
def ganho_info(df, atributo):
    values, counts = np.unique(df[atributo], return_counts=True)
    prob = counts/counts.sum()
    entropy2 = 0
    for i in range(len(values)):
        cases = df.loc[df[atributo] == values[i]]
        entropy2 += counts[i]*entropy(cases)/counts.sum()
    ganho = entropy(df) - entropy2
    return ganho, atributo
    
        

### Escolha dos melhores atributos
- Utilizando a função de ganho de informação, foi possível determinar a ordem da árvore de decisão
- Assim, podemos perceber que a ordem de prioridade dos atributos foi: 'Occupation', 'birthday' e 'Gender'

In [91]:
def melhor_atributo(df):
    order = []
    for column in df.columns:
        if(column != 'Rating' and column != 'MovieID'):
#             print(ganho_info(df, column))
            order.append(ganho_info(df, column))
    order.sort(reverse=True)
#     print(order)
melhor_atributo(df)

## Algoritmo ID3

- Utiliza dos atributos com melhor ganho de informação para ficarem acima na árvore
- Caso não seja encontrado um caso em que tenha rating do filme para os atributos desejados, a classificação será a média truncada do filme

In [92]:
def ID3(df, id_movie, ocupacao, birthday, gender):
    #Para apenas valores de id_movie:
    valores = df.loc[df['MovieID']==id_movie]
    #Caso não tenha uma classificação com todos os atributos desejados, a classificação será a média truncada da classificação do filme
    classificacao = round(valores['Rating'].sum()/len(valores['Rating']))
    valores = valores.loc[valores['Occupation']==ocupacao]
    valores = valores.loc[valores['birthday']==birthday]
    valores = valores.loc[valores['Gender']==gender]
    if(len(valores)!=0):
        classificacao = round(valores['Rating'].sum()/len(valores['Rating']))
    return classificacao
    
ID3(df, 1193, 1, '25', 'M')

4

## Classificação a priori - Média Truncada

In [93]:
def media_truncada(df, id_movie):
    valores=df.loc[df['MovieID']==id_movie]
    media = round(valores['Rating'].sum()/len(valores['Rating']))
#     print(media)
    return media

ids = np.unique(df['MovieID'])
for i in ids:
    media_truncada(df, i)

## Análise Comparativa
- Foram escolhidos os seguintes filmes com as classificações ditadas pelo grupo:
    - MovieID:1 | Title: Toy Story (1995) | Classificação: 4
    - MovieID:154 | Belle de jour (1967) | Classificação: 2
    - MovieID:260 | Star Wars: Episode IV - A New Hope (1977) | Classificação: 5
    - MovieID:255 | Jerky Boys, The (1994) | Classificação: 3
    - MovieID:480 | Jurassic Park (1993) | Classificação: 3
    - MovieID:520 | Robin Hood: Men in Tights (1993) | Classificação: 4
    - MovieID:592 | Batman (1989) | Classificação: 4
    - MovieID:1221 | Godfather: Part II, The (1974) | Classificação: 4
    - MovieID:1240 | Terminator, The (1984) | Classificação: 5
    - MovieID:3945 | Digimon: The Movie (2000) | Classificação: 5
    

## 1: Classificador por ID3

### Taxa de Acerto

In [118]:
acerto = []
movies_ids = [1, 154, 260, 255, 480, 520, 592, 1221, 1240, 3945]
classificacoes = [4, 2, 5, 3, 3, 4, 4, 4, 5, 5]
# print(ID3(df, 1, 4, '18', 'M')
# print(movies_ids)
num_acertos = 0
for i in range(10):
    if(classificacoes[i]==ID3(df, movies_ids[i], 4, '18', 'M')):
        num_acertos += 1
        acerto.append(1)
    else:
        acerto.append(0)
taxa_acerto_id3 = num_acertos/10
print(taxa_acerto_id3)

0.4


### Erro Quadrático Médio

In [115]:
erro = []
erro_total = 0
for i in range(10):
    erro_i = (classificacoes[i]-ID3(df, movies_ids[i], 4, '18', 'M'))^2
    erro_total+=erro_i
erro_quad_medio = erro_total/10
print(erro_quad_medio)

1.2


## 2: Classificador A priori

### Taxa de Acerto

In [119]:
acerto = []
movies_ids = [1, 154, 260, 255, 480, 520, 592, 1221, 1240, 3945]
classificacoes = [4, 2, 5, 3, 3, 4, 4, 4, 5, 5]
num_acertos = 0
for i in range(10):
    if(classificacoes[i]==media_truncada(df, movies_ids[i])):
        num_acertos += 1
        acerto.append(1)
    else:
        acerto.append(0)
taxa_acerto_priori = num_acertos/10
print(taxa_acerto_priori)

0.3


### Erro Quadrático Médio

In [116]:
erro = []
erro_total = 0
for i in range(10):
    erro_i = (classificacoes[i]-media_truncada(df, movies_ids[i]))^2
    erro_total+=erro_i
erro_quad_medio = erro_total/10
print(erro_quad_medio)

1.7


## Estatística Kappa

In [120]:
Kappa = (taxa_acerto_id3 - taxa_acerto_priori)/(1-taxa_acerto_priori)
print(Kappa)

0.1428571428571429


## Conclusão

O trabalho solicitado foi bastante condizente com a teoria apresentada em sala de aula, propiciando a aplicação dos conceitos teóricos básicos, essenciais para o desenvolvimento da matéira de ciência de dados.
A execução do trabalho foi complexa, não pela dificuldade de entender o embasamento teórico que permeia os conceitos aplicados, mas sim pela dificuldade de aplicar os algoritmos necessários na linguagem computacional.
O grupo gestaria de deixar como sugestão desenvolver, seja em sala de aula, por meio de vídeo-aulas, ou ainda por meio de materiais escritos no google classroom, a parte prática da disciplina, mostrando como aplicar os conceitos na linguagem computacional, conforme é exigido nas práticas de laboratório.

## Implementação

A implementação do trabalho foi realizada em python e a IDE utilizada foi o Jupyter Notebook.